In [104]:
import pickle
import numpy as np
import qiskit
from qiskit.transpiler import Layout
from qiskit.opflow.state_fns import DictStateFn
from qiskit.opflow import I, X, Y, Z, CircuitOp, StateFn
from qiskit import QuantumCircuit
from qiskit_ibm_provider import IBMProvider
# IBMProvider.save_account("700663709ad0eadb905450245f20e3e363ef30b27fb5971bc4544567711e8458b187869d5882e5bc2134ec6a0ed07014ae50e0b344b8b1556022d0de348f4447", overwrite=True)
# provider = IBMProvider()
# print(provider.backends())
import math
from typing import List

import argparse
import os
from functools import partial

import numpy as np
import networkx as nx
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.utils import QuantumInstance
from qiskit.opflow import AerPauliExpectation, PauliExpectation, StateFn, DictStateFn, CircuitStateFn, Z
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import Maxcut, SKModel
# from qiskit_optimization.translators import from_docplex_mp
from qiskit.utils import algorithm_globals
# from docplex.mp.model import Model
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
from qiskit.quantum_info.analysis import hellinger_fidelity

from qiskit.converters import circuit_to_dag

from oscar.utils import maxcut_obj, obj_from_statevector, get_adjacency_matrix

In [105]:

def get_energy(G, qc: QuantumCircuit=None, cnts: dict=None):
    # print(qc)
    # qc.save_statevector()
    shots = 1024
    obj = partial(maxcut_obj, w=get_adjacency_matrix(G))
    if cnts == None:
        backend = AerSimulator(method="statevector")
        cnts = backend.run(qc, shots=shots).result().get_counts()

    obj_val = 0.0
    for bitstring, cnt in cnts.items():
        bitstring_int = np.array(list(map(int, list(bitstring))))
        obj_val += obj(bitstring_int) * (cnt / shots)
        
    return obj_val

In [106]:

parser = argparse.ArgumentParser()
parser.add_argument("--problem", type=str, default="maxcut")
parser.add_argument("-n", type=int, default=16)
parser.add_argument("-p", type=int, default=1)
parser.add_argument("-s", "--seed", type=int, required=True)
parser.add_argument("-b", "--backend", type=str, default="sv")
parser.add_argument("--ansatz", type=str, default="qaoa")
parser.add_argument("--cpu", default=False, action="store_true")
parser.add_argument("--no-aer", dest="aer", default=True, action="store_false")
parser.add_argument("--noise", type=str, default="ideal")
parser.add_argument("--p1", type=float, default=0.001)
parser.add_argument("--p2", type=float, default=0.005)
parser.add_argument("--beta-steps", type=int, default=50)
parser.add_argument("--gamma-steps", type=int, default=100)


_StoreAction(option_strings=['--gamma-steps'], dest='gamma_steps', nargs=None, const=None, default=100, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [107]:
def load_ibm_data(ibm_id: int, seed: int, shots: int):
    if shots == 2048:
        path = f"figs/ibm/IBM_Exp_2048/IBM-M{ibm_id}/Experimental_Data,Seed-{seed}P=1circ_param_lst_2048.pkl"
        recon_save_dir = "figs/ibm/IBM_Exp_2048"
    elif shots == 1024:
        path = f"figs/ibm/Experiments/IBM-M{ibm_id}/Experimental_Data,Seed-{seed}P=1circ_param_lst.pkl"
        recon_save_dir = "figs/ibm/Experiments"
    else:
        raise ValueError("shots must be 1024 or 2048")
        
    print(f"read from IBM={ibm_id}, seed={seed}")
    file = open(path, 'rb')
    data = pickle.load(file)
    file.close()

    cnts_list = data[0]
    transpiled_circ = data[1][0]

    return cnts_list, transpiled_circ, recon_save_dir


def diff(a: dict, b: dict, type: str):
    diff = 0.0
    if type == 'kl':
        P = dict2arr(a)
        Q = dict2arr(b)
        diff = kl_div(P, Q)
    elif type == 'hellinger_fidelity':
        diff = hellinger_fidelity(a, b)
    elif type == 'tvd':
        P = dict2arr(a)
        Q = dict2arr(b)
        diff = total_variation_distance(P, Q)
    return diff
    # return norm_between_two_dist_in_dict_form(a, b)


def kl_div(P, Q):
    return np.sum(np.where(P * Q != 0, P * np.log(P / Q), 0))
    """ Epsilon is used here to avoid conditional code for
    checking that neither P nor Q is equal to 0. """
    # epsilon = 0.00001

    # # # You may want to instead make copies to avoid changing the np arrays.
    # P = P + epsilon
    # Q = Q + epsilon

    # divergence = np.sum(P * np.log(P / Q))
    # return divergence


def dict2arr(dist_dict: dict):
    num_qubits = len(next(iter(dist_dict)))
    arr = np.zeros(2 ** num_qubits)
    shots = np.sum(list(dist_dict.values()))
    # print("shots=", shots)
    for bitstring, cnt in dist_dict.items():
        idx = int(bitstring, 2)
        arr[idx] = 1.0 * cnt / shots
    assert np.isclose(arr.sum(), 1.0)
    return arr


def norm_between_two_dist_in_dict_form(a: dict, b: dict):
    norm = 0.0
    intersection = a.keys() & b.keys()
    for key in intersection:
        norm += (a[key] - b[key])**2

    for key in a.keys() - b.keys():
        norm += a[key] ** 2

    for key in b.keys() - a.keys():
        norm += b[key] ** 2

    return norm


def total_variation_distance(p, q):
    return 0.5 * np.sum(np.abs(p - q))

MID_TO_DEVICE_NAME = {
    1: 'ibm_perth',
    2: 'ibm_lagos',
}
def get_fname(mid: int, seed: int, noise: str):
    bs = 50
    gs = 100
    n = 6
    p = 1
    problem = "maxcut"

    assert noise in ['ideal_sim', 'noisy_sim', 'real']
    device_name = MID_TO_DEVICE_NAME[mid]
    fname = f"{problem}-{device_name}-{noise}-{n=}-{p=}-{seed=}-{bs}-{gs}"
    return fname

In [130]:
def generate_landscape_from_transpiled_circ(args, ibm_id: int, qcs: QuantumCircuit, exp_val_type: str):
    print(args.__dict__)
    n = args.n
    p = args.p
    seed = args.seed
    backend_config = args.backend.lower()
    noise = args.noise.lower()

    device = "CPU" if args.cpu else "GPU"
    backend_config += f" {noise}"

    if noise == 'ideal_sim':
        noise_model = None
    elif noise == 'noisy_sim':
        provider = IBMProvider()
        ibm_device = MID_TO_DEVICE_NAME[ibm_id]
        noise_model = NoiseModel.from_backend(provider.get_backend(ibm_device))
    else:
        raise NotImplementedError(f"Noise {noise} not implemented yet")

    backend = AerSimulator(
        method="statevector",
        device=device,
        noise_model=noise_model,
        fusion_enable=args.problem == "maxcut" or noise != "depolar" or n < 17,
    )
    backend_config = backend_config.replace(" ", "-")

    # --------------- define the problem ------------

    G = nx.random_regular_graph(3, n, seed)
    problem = Maxcut(G).to_quadratic_program()
    H, offset = problem.to_ising()

    algorithm_globals.random_seed = seed

    # TODO: you might need to modify this to config IBM backend
    # quantum_instance = QuantumInstance(
    #     backend=backend,
    #     seed_simulator=seed,
    #     seed_transpiler=seed,
    #     optimization_level=None,
    # )

    # algorithm = QAOA(
    #     SPSA(),
    #     reps=p,
    #     quantum_instance=quantum_instance,
    #     expectation=AerPauliExpectation() if args.aer else None,
    # )
    # algorithm._check_operator_ansatz(H)
    # energy_evaluation, expectation = algorithm.get_energy_evaluation(
    #     H, return_expectation=True
    # )

    # ------------- transpile ----------------
    # def energy_evaluation_transpiled(parameters: np.ndarray) -> float:
    #     qc = quantum_instance.transpile(algorithm.ansatz.bind_parameters(parameters))[0]
    #     observable_meas = expectation.convert(StateFn(H, is_measurement=True))
    #     expect_op = observable_meas.compose(CircuitStateFn(qc)).reduce()
    #     sampled_expect_op = algorithm._circuit_sampler.convert(expect_op)
    #     return np.real(sampled_expect_op.eval())

    # energy_evaluation = energy_evaluation_transpiled


    dirpath = f"figs/ibm/{args.ansatz}/{args.problem}/{backend_config}-{p=}"
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

    # savepath = f"{dirpath}/{args.problem}-{backend_config}-{n=}-{p=}-{seed=}-{args.beta_steps}-{args.gamma_steps}-IBM{ibm_id}-transpiled-{exp_val_type}"
    savepath = f"{dirpath}/{get_fname(ibm_id, seed, noise)}"

    if os.path.exists(f"{savepath}.npz"):
        data = np.load(
            f"{savepath}.npz",
            allow_pickle=True
        )

        return data['cnts_list'], data['data'], G, offset, H, dict(data)
    

    # ------------- prepare parameters and execute the circuit --------------
    data = []
    beta_bound = np.pi / 4 / p
    gamma_bound = np.pi / 2 / p

    grid = np.array(
        np.meshgrid(
            *np.linspace([-beta_bound] * p, [beta_bound] * p, args.beta_steps, axis=1),
            *np.linspace([-gamma_bound] * p, [gamma_bound] * p, args.gamma_steps, axis=1),
            indexing="ij",
        )
    )
    
    tmp = grid.transpose((*range(1, 2 * p + 1), 0)).reshape(-1, 2 * p)
    observable_meas = H
    # observable_meas = Z^6
    cnts_list = []
    for qc in tqdm(qcs): # transpiled circuit
        if exp_val_type == 'H':
            job = backend.run(qc, shots=2048)
            cnts = job.result().get_counts()
            # for bitstring, cnt in cnts.items():
            #     cnts[bitstring] = np.sqrt(cnt / 1024.0)
            cnts_list.append(cnts)
            qc = DictStateFn(job.result())
            exp_val = (~qc @ observable_meas @ qc).eval().real
        elif exp_val_type == 'graph':
            exp_val = get_energy(graph, qc)
        elif exp_val_type == 'test':
            exp_val = 0
        else:
            raise NotImplementedError(f"exp_val_type {exp_val_type} not implemented yet")
            
        data.append(exp_val)

        # observable_meas = expectation.convert(StateFn(H, is_measurement=True))
        # expect_op = observable_meas.compose(qc) # .reduce()
        # sampled_expect_op = algorithm._circuit_sampler.convert(expect_op)
        # energy = np.real(sampled_expect_op.eval())
        # energy = energy_evaluation(params) # ! execute the circuit
        # data.append(energy) 

    data = np.array(data).reshape(([args.beta_steps] * p) + ([args.gamma_steps] * p))

    # ----------- save the data ---------
    data_dict = {
        "data": data,
        "offset": offset,
        "beta_bound": beta_bound,
        "gamma_bound": gamma_bound,
        "grid": grid,
        "cnts_list": cnts_list
    }

    np.savez_compressed(
        savepath,
        # data=data,
        # offset=offset,
        # beta_bound=beta_bound,
        # gamma_bound=gamma_bound,
        # grid=grid,
        # cnts_list=cnts_list,
        **data_dict,
        **args.__dict__,
    )

    return cnts_list, data, G, offset, H, data_dict


In [131]:
def get_ls(G, cnts_list):
    ls = []
    for cnts in cnts_list:
        pt = get_energy(G, None, cnts)
        ls.append(pt)

    return np.array(ls).reshape((50, 100))

def get_ls_by_H(cnts_list, H):
    ls = []
    
    for cnts in cnts_list:
        primitive = {
            bstr: math.sqrt(shots / sum(cnts.values())) for (bstr, shots) in cnts.items()
        }
        qc = DictStateFn(primitive)
        exp_val = (~qc @ H @ qc).eval().real # < \psi | H | \psi >, compose()
        ls.append(exp_val)

    return np.array(ls).reshape((50, 100))

In [134]:
# check difference between counts

def vis():
    fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(15, 12))
    axs = axs.reshape(-1)
    idx = 0
    i = 0
    axs[i].imshow(ibm_ls)
    axs[i].set_title("IBM, bstr")

    axs[i+1].imshow(ibm_H_ls)
    axs[i+1].set_title("IBM, H")

    axs[i+2].imshow(sim_cnt_ls)
    axs[i+2].set_title("Sim., bstr")
    
    im = axs[i+3].imshow(sim_H_ls)
    axs[i+3].set_title("Sim., H")
    
    vis_path = f"{save_path}-vis"
    # if not os.path.exists(vis_path):
    #     os.makedirs(vis_path)
    plt.savefig(vis_path)

    plt.show()

def rebuild_ibm_landscapes(ibm: int, seed: int, tcircs: List[QuantumCircuit]):
    is_cnt_cx = False
    is_graph_cost = True

    
    if is_cnt_cx:
        n_cx = 0
        for tcirc in tcircs:
            dag = circuit_to_dag(tcirc)
            # print(tcirc)
            for node in dag.topological_op_nodes():
                if node.name == 'cx':
                    n_cx += 1

        n_cx /= len(tcircs) * 1.0
        print(n_cx)
    
    # for noise in ['ideal_sim', 'noisy_sim']:
    for noise in ['ideal_sim']:
        cmd = f"-n 6 -p 1 --seed {seed} --noise {noise} --cpu --backend {MID_TO_DEVICE_NAME[ibm]} --ansatz qaoa"
        args = parser.parse_args(list(cmd.split(' ')))
        sim_cnts_list, ls, G, offset, H, data_dict = generate_landscape_from_transpiled_circ(args, ibm, tcircs, 'H')
        
        if False:
            norm = 0.0
            for a, b in zip(ibm_cnts_list, sim_cnts_list):
            # for a, b in zip(sim_cnts_list, ibm_cnts_list):
                norm += diff(a, b, 'hellinger_fidelity')
                # norm += diff(a, b, 'tvd')
            norm /= len(ibm_cnts_list)
            print("norm =", norm)
    
        # if False:
        ibm_ls = - get_ls(G, ibm_cnts_list)
        ibm_H_ls = get_ls_by_H(ibm_cnts_list, H)

        sim_cnt_ls = - get_ls(G, sim_cnts_list)
        sim_H_ls = get_ls_by_H(sim_cnts_list, H)
        # sim_H_ls = (ls + offset)
        print(f"offset={offset}")
        
        # save_path = f"{save_dir}/ls-M-{ibm}-seed-{seed}-{noise}"
        save_dir = f"figs/ibm/qaoa/maxcut"
        save_path = f"{save_dir}/{get_fname(ibm, seed, 'real')}"
        
        np.savez_compressed(
            save_path,
            data=ibm_H_ls,
            offset=data_dict["offset"],
            beta_bound=data_dict["beta_bound"],
            gamma_bound=data_dict["gamma_bound"],
            grid=data_dict["grid"],
            cnts_list=data_dict["cnts_list"],

            # ibm_bstr_ls=ibm_ls,
            # ibm_H_ls=ibm_H_ls,

            # sim_bstr_ls=sim_cnt_ls,
            # sim_H_ls=sim_H_ls
        )

        lss = {
            "ibm_bstr_ls": ibm_ls,
            "ibm_H_ls": ibm_H_ls,
            "sim_bstr_ls": sim_cnt_ls,
            "sim_H_ls": sim_H_ls
        }

        return lss    



In [115]:
ibm = 1
seed = 1
shots = 2048
ibm_cnts_list, tcircs, save_dir = load_ibm_data(ibm, seed, shots)

read from IBM=1, seed=1


In [135]:
lss = rebuild_ibm_landscapes(ibm, seed, tcircs)

{'problem': 'maxcut', 'n': 6, 'p': 1, 'seed': 1, 'backend': 'ibm_perth', 'ansatz': 'qaoa', 'cpu': True, 'aer': True, 'noise': 'ideal_sim', 'p1': 0.001, 'p2': 0.005, 'beta_steps': 50, 'gamma_steps': 100}
offset=-4.5


In [139]:
# # check there is no difference between ibm_H_ls

def check(mid: int, seed: int, noise: str):
    fname = get_fname(mid, seed, noise)
    a = lss['ibm_H_ls']
    # a = np.load(f"figs/ibm/IBM_Exp_2048/{fname}.npz")['data']
    # ideal = np.load("figs/ibm/IBM_Exp_2048/ls-M-1-seed-1-ideal.npz")['ibm_H_ls']
    # perth = np.load("figs/ibm/IBM_Exp_2048/ls-M-1-seed-1-ibm_perth.npz")['ibm_H_ls']
    # b = np.load(f"figs/ibm/qaoa/maxcut/{MID_TO_DEVICE_NAME[mid]}-{noise}-p=1/{fname}.npz")['data']
    b = np.load(f"figs/ibm/qaoa/maxcut/{fname}.npz", allow_pickle=True)['data']

    print(np.abs(a - b).sum())
    # print(np.allclose(a, b))

check(ibm, seed, 'real')

0.0


In [ ]:
# IBM 1: 32, 1.2%
# IBM 2: largos, 32, 1%
# IBM 3: aslow, coherence time is 1/3 of 1 & 2

import random
norm = 0.0
n_tries = 5000
n_qubits = 6
shots = 1024
for i in range(n_tries):

    aa = [random.randint(0, 2 ** n_qubits - 1) for _ in range(shots)]
    bb = [random.randint(0, 2 ** n_qubits - 1) for _ in range(shots)]

    a = np.zeros(2 ** n_qubits)
    b = np.zeros(2 ** n_qubits)

    for i in aa:
        # print(i)
        a[i] += 1
    
    for i in bb:
        b[i] += 1
    
    # a = np.maximum(a, 0)
    # b = np.maximum(b, 0)
    # a += a.min()
    # b += b.min()

    a /= shots
    b /= shots
    norm += kl_div(a, b)

norm /= n_tries

print(norm)

In [ ]:
fig = plt.figure(figsize=(6, 5))
sfs = [0.01, 0.03, 0.05, 0.07, 0.09]
for ibm in ibm_ids:
    for seed in seeds: # three instances
        if ibm == 3 and seed == 1:
            continue

        path = f"figs/grid_search_recon/ibm/machine-{ibm}/recon_error_mid={ibm}-n=6-seed={seed}-sfs=[0.01 0.03 0.05 0.07 0.09]-error=NRMSE.npz"
        recon_error_data = np.load(path, allow_pickle=True)
        sfs = recon_error_data['sfs']
        mses = recon_error_data['mses']
        plt.plot(sfs, mses, label=f"M{ibm}, seed {seed}")
plt.ylabel("NRMSE")
plt.xlabel("Sampling fraction")
# plt.suptitle("NRMSE")
plt.legend()